# Mixture of Linear Regressions

In this note, we will solve the mixture of linear regressions problem using the moment method.

This block contains some $\LaTeX$ macros.
$\newcommand{\E}{\mathbb{E}}$
$\renewcommand{\Re}{\mathbb{R}}$
$\newcommand{\oft}[1]{{^{(#1)}}}$
$\newcommand{\oftt}[2]{{^{(#1)#2}}}$

In [1]:
import util
from util import *
import models
from models.LinearRegressions import LinearRegressionsMixture
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display, Markdown, Math
from operator import mul
import sympy as sp
sp.init_printing()

Let $w_1, ..., w_K \in \Re^D$ be a set of $K$ regression coefficients. Let $x_1, ..., x_N \in \Re^D$ be a given set of data points and let $y_n = \sum_{k} \delta_k w_k^\top x_n + \epsilon$ be observed responses. Our objective is to recover $(w_k)$.

## Toy Example

Let's construct a toy example for the rest of this document.

In [2]:
K, D = 2, 2
np.random.seed(0)
pis = np.array([0.4, 0.6])
ws = np.array([[0.75, 0.25], [0.4, 0.9]])
model = LinearRegressionsMixture.generate(K, D, betas = ws, weights = pis, cov =  1.0)
print("True parameters:")
print(model.weights)
print(model.betas)
print(model.sigma)

True parameters:
[ 0.4  0.6]
[[ 0.75  0.25]
 [ 0.4   0.9 ]]
[[ 1.  0.]
 [ 0.  1.]]


# Noiseless, Infinite Data Setting

In this scenario, assume that $\epsilon = 0$ and that we observe expected moments of the data. Consider the moments:
\begin{align}
\E[x^\alpha y^b] 
&= \sum_k \pi_k \E[x^\alpha y^b | \delta_k = 1] \\
&= \sum_k \pi_k \E[x^\alpha [w_k^\top x]^b] \\
&= \sum_k \pi_k \E[x^\alpha \sum_{\beta \in p(b)} w_k^\beta x^\beta] \\
&= \sum_{\beta \in p(b)} \E_\pi[w^\beta] \E[x^{\alpha + \beta}],
\end{align}
where $p(b)$ are the $d$-partitions of $b$. Note that $\E[x^{\alpha + \beta}]$ are observable quantities, and thus this simply represents a mixture over the polynomials with terms $w_k^\beta$.

In [3]:
def evaluate_mixture(ws, pis, beta):
    """
    Compute E_\pi[w^beta]
    """
    return sum(pi * util.monomial(w, beta) for w, pi in zip(ws.T, pis))
    
def compute_exact_y_moments(ws, pis, moments_x, alpha, b):
    """
    Compute the exact moments E[x^a y^b] using coefficients ws, pis and moments_x.
    """
    D, K = ws.shape
    ret = 0.
    coeffs = sp.ntheory.multinomial_coefficients(D,b)
    for beta in partitions(D, b):
        ret += coeffs[beta] * evaluate_mixture(ws, pis, beta) * moments_x[tuple_add(alpha, beta)]
    return ret

def describe_moment_polynomial(R, moments_x, moment_y, alpha, b):
    """
    Computes the moment polynomial for E[x^alpha, y^b]
    """
    D = len(R.symbols)
    w = R.symbols
    expr = -moment_y
    coeffs = sp.ntheory.multinomial_coefficients(D,b)
    for beta in partitions(D, b):
        expr += coeffs[beta] * util.monomial(w, beta) * moments_x[tuple_add(alpha, beta)]
    return expr

In [4]:
# Example 
def double_factorial(n): 
    return reduce(mul, xrange(n, 0, -2)) if n > 0 else 1
def gaussian_moments(sigma, d):
    """
    E[x^d] where x is standard gaussian with sigma
    """
    if d == 0: return 1
    elif d % 2 == 0: return double_factorial(d-1) * sigma**d
    else: return 0
def expected_gaussian_moments(sigma, alphas):
    return {alpha : prod(gaussian_moments(sigma, a) for a in alpha) for alpha in alphas}
def expected_uniform_moments(alphas):
    return {alpha : 1. for alpha in alphas}
def expected_moments_y(ws, pis, moments_x, alphabs):
    return {(alpha, b) : compute_exact_y_moments(ws, pis, moments_x, alpha, b) for alpha, b in alphabs}

In [5]:
R, _ = sp.xring(['w%d'%d for d in xrange(D)], sp.RR, sp.grevlex)

deg_b, deg_x = 3, 2
sigma = model.sigma[0,0]
alphas = list(dominated_elements((deg_x for _ in xrange(D))))
alphabs = [(alpha, b) for alpha in alphas for b in xrange(1,deg_b+1)]
alphas_ = list(dominated_elements((deg_x + deg_b for _ in xrange(D))))
moments_x = expected_gaussian_moments(sigma, alphas_)
moments_y = expected_moments_y(ws, pis, moments_x, alphabs)
#display(moments)

In [6]:
display(moments_y)

In [7]:
def get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b):
    constrs = []
    for b in xrange(1, deg_b+1):
        for alpha in util.dominated_elements((deg_x for _ in xrange(D))):
            constrs.append( describe_moment_polynomial(R, moments_x, moments_y[(alpha, b)], alpha, b) )
    return constrs

With this machinery, we can compute the moment polynomials required for the moment method magic!

In [8]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)

constrs = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
M = MomentMatrix(3, R.symbols, morder='grevlex')
sol = solvers.solve_generalized_mom_coneqp(M, constrs, 3)
display( model.betas)
display(extractors.extract_solutions_lasserre(M, sol['x'], Kmax=2))

     pcost       dcost       gap    pres   dres
 0:  3.3331e-01 -1.1934e+00  4e+01  4e+00  8e+00
 1:  1.9518e-01 -3.2675e+00  7e+00  8e-01  1e+00
 2:  1.4298e-01 -1.0911e+00  1e+00  1e-01  2e-01
 3:  3.5067e-02 -1.2531e-01  2e-01  1e-02  2e-02
 4:  5.2528e-03 -1.8015e-02  2e-02  9e-04  2e-03
 5:  8.6614e-04 -2.3142e-03  3e-03  9e-05  2e-04
 6:  2.5471e-04 -8.0417e-05  3e-04  9e-07  2e-06
 7:  5.9055e-05  2.3865e-06  6e-05  1e-07  2e-07
 8:  4.0805e-05  2.3551e-05  2e-05  1e-08  2e-08
 9:  3.0154e-05  2.7546e-05  3e-06  1e-09  2e-09
10:  2.9128e-05  2.8320e-05  8e-07  2e-10  3e-10
11:  2.8647e-05  2.8507e-05  1e-07  2e-11  4e-11
12:  2.8587e-05  2.8549e-05  4e-08  4e-12  7e-12
Optimal solution found.


array([[ 0.75,  0.25],
       [ 0.4 ,  0.9 ]])

{w0: array([ 0.25,  0.75]), w1: array([ 0.9,  0.4])}

In [9]:
display(constrs)

## With real samples

Let's try to solve the problem with generated samples.


In [10]:
def compute_expected_moments(xs, alphas):
    moments = {alpha : 0. for alpha in alphas}
    for alpha in alphas:
        m = monomial(xs.T, alpha)
        moments[alpha] = m if isinstance(m,float) else float(m.mean())
    return moments

def compute_expected_moments_y(ys, xs, alphabs):
    moments = {(alpha, b) : 0. for alpha, b in alphabs}
    for alpha, b in alphabs:
        moments[(alpha,b)] = float((monomial(xs.T, alpha) * ys**b).mean())
    return moments

In [11]:
from mompy.core import MomentMatrix
import mompy.solvers as solvers; reload(solvers)
import mompy.extractors as extractors; reload(extractors)
from util import fix_parameters, column_rerr
totalerror = 0
for t in range(1):
    xs, ys = model.sample(1e6)
    moments_x = compute_expected_moments(xs, alphas_)
    moments_y = compute_expected_moments_y(ys, xs, alphabs)

    constrs_sdp = get_constraint_polynomials(moments_y, moments_x, deg_x, deg_b)
    M = MomentMatrix(3, R.symbols, morder='grevlex')
    solsdp = solvers.solve_generalized_mom_coneqp(M, constrs_sdp, None)
    #sol = solvers.solve_moments_with_convexiterations(M, constrs, 3)
     
    betas_lassare = extractors.extract_solutions_lasserre(M, solsdp['x'], Kmax=2)
    betas_lassare_array = np.array([betas_lassare[R.symbols[0]], betas_lassare[R.symbols[1]]])
        
    M_ = closest_permuted_matrix(model.betas.T, betas_lassare_array.T)
    
    display(M_)
    display('the true parameters')
    display(model.betas.T)
    totalerror += column_aerr(M_, model.betas)
print totalerror
    

     pcost       dcost       gap    pres   dres
 0:  3.3368e-01 -1.1933e+00  4e+01  4e+00  8e+00
 1:  1.9538e-01 -3.2668e+00  7e+00  8e-01  1e+00
 2:  1.4312e-01 -1.0906e+00  1e+00  1e-01  2e-01
 3:  3.5230e-02 -1.2495e-01  2e-01  1e-02  2e-02
 4:  5.4292e-03 -1.7808e-02  2e-02  9e-04  2e-03
 5:  1.0384e-03 -2.1377e-03  3e-03  9e-05  2e-04
 6:  4.2657e-04  9.2214e-05  3e-04  9e-07  2e-06
 7:  2.3063e-04  1.7427e-04  6e-05  1e-07  2e-07
 8:  2.1266e-04  1.9500e-04  2e-05  1e-08  2e-08
 9:  2.0183e-04  1.9907e-04  3e-06  1e-09  2e-09
10:  2.0084e-04  1.9988e-04  1e-06  3e-10  5e-10
11:  2.0028e-04  2.0015e-04  1e-07  3e-11  6e-11
12:  2.0024e-04  2.0021e-04  3e-08  4e-12  8e-12
Optimal solution found.


array([[ 0.75 ,  0.404],
       [ 0.252,  0.903]])

'the true parameters'

array([[ 0.75,  0.4 ],
       [ 0.25,  0.9 ]])

0.153823872173
